### **单机多卡并行**
- 一台机器可以安装多个 GPU （1-16）
- 在训练和预测时，我们将一个小批量计算切分到多个 GPU 上来达到加速目的
- 常用切分方案有
  - 数据并行
  - 模型并行
  - 通道并行（数据 + 模型 并行）

### **数据并行 vs 模型并行**
- 数据并行：将小批量分成 n 块，每个 GPU 拿到完整参数计算一块数据的梯度
  - 通常性能更好
- 模型并行：将模型分成 n 块，每个 GPU 拿到一块模型计算它的前向和反向结果
  - 通常用于模型大到单个 GPU 放不下

### **[数据并行](https://www.bilibili.com/video/BV1vU4y1V7rd/?share_source=copy_web&vd_source=a7ae9163cb2cd121bfd86ea1f4ecd2ef&t=225)**

### **总结**
- 当一个模型能用单卡计算时，通常使用数据并行拓展到多卡上
- 模型并行则用在超大模型上

-----------------------------
### **QA**
- 数据并行时 GPU 大小不一样也没关系，我们分配不同的样本就OK

- 多个 GPU 上不同的并行方式

![在多个GPU上并行化。从左到右：原始问题、网络并行、分层并行、数据并行](img/splitting.svg)

- 有多种方法可以在多个 GPU 上拆分深度网络的训练。**拆分可以在层之间、跨层或跨数据上实现。前两者需要对数据传输过程进行严格编排，而最后一种则是最简单的策略**。

----------------------------------------

## 数据并行性

假设一台机器有$k$个GPU。
给定需要训练的模型，虽然每个GPU上的参数值都是相同且同步的，但是每个GPU都将独立地维护一组完整的模型参数。
例如，下图演示了在$k=2$时基于数据并行方法训练模型。

![利用两个GPU上的数据，并行计算小批量随机梯度下降](img/data-parallel.svg)

一般来说，$k$个GPU并行训练过程如下：

* 在任何一次训练迭代中，给定的随机的小批量样本都将被分成$k$个部分，并均匀地分配到GPU上；
* 每个GPU根据分配给它的小批量子集，计算模型参数的损失和梯度；
* 将$k$个GPU中的局部梯度聚合，以获得当前小批量的随机梯度；
* 聚合梯度被重新分发到每个GPU中；
* 每个GPU使用这个小批量随机梯度，来更新它所维护的完整的模型参数集。


在实践中请注意，当在$k$个GPU上训练时，需要扩大小批量的大小为$k$的倍数，这样每个GPU都有相同的工作量，就像只在单个GPU上训练一样。
因此，在16-GPU服务器上可以显著地增加小批量数据量的大小，**同时可能还需要相应地提高学习率。还请注意，批量规范化也需要调整**，例如，为每个GPU保留单独的批量规范化参数。